In [21]:
!pip install wikipedia-api
import wikipediaapi
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import json
!wget https://raw.githubusercontent.com/SEREMM/some_funtions/main/some_funcs.py
from some_funcs import read_gsheet_shared
class CloseException(Exception):
  pass

--2023-11-14 02:22:21--  https://raw.githubusercontent.com/SEREMM/some_funtions/main/some_funcs.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491 [text/plain]
Saving to: ‘some_funcs.py.1’

some_funcs.py.1     100%[===================>]     491  --.-KB/s    in 0s      

2023-11-14 02:22:21 (42.1 MB/s) - ‘some_funcs.py.1’ saved [491/491]



## box basics

In [22]:
class Boxers_logger_basics:
  def authorization(self, old_data, intend, new_data):
    autho = input(f'Proceed with {intend}? (y/n):')
    if autho == 'y':
      new_data.to_csv(old_data, index=False)
      return 1
    else:
      return 0

  def json_log(self, old_data, new_data):
    try:
      with open(old_data, 'r') as json_f:
        data = json.load(json_f)
      data.append(new_data)
    except FileNotFoundError:
      data = [new_data]
    with open(old_data, 'w') as json_f:
      json.dump(data, json_f, indent=4)
      print(f'Json data appended to {old_data}\n')

  def delete_row(self, file_to_get_modifed, id, column):
    try:
      data = pd.read_csv(file_to_get_modifed)# checar no de problemas el archivo del cual borrar
      filtered_df = data[data[column] != id]
      aut = self.authorization(file_to_get_modifed, 'delete record', filtered_df)
      if aut == 1:
        print(f'Row deleted successfully\n')
      else:
        print('Authorization to delete denied\n')
    except Exception as e:
      print(f"Error deleting row: {str(e)} - maybe id mismatch\n")

  def closer(self, message):
    res = input(f'{message}')
    if res == 'c':
      raise CloseException()
    else:
      return res

## boxers info

In [23]:
class Boxers_info(Boxers_logger_basics):
  '''
  Class to modify, append, search or delete records in the boxers info file.
  '''
  def __init__(self, filename, json_file):
    '''
    Constructor method.
    Parameters:
      filename: str with the name of the boxers file.
      json_file: str with the name of the json file to save scrapped data.
    '''
    self.filename = filename
    self.json_file = json_file

  def getPage(self, url):
    session = requests.Session()
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}
    try:
        req = session.get(url, headers=headers)
    except requests.exceptions.RequestException:
        return None
    bs = BeautifulSoup(req.text, 'html.parser')
    return bs

  def info_box_page(self, boxer):
    boxer1 = boxer.replace(' ','-')
    raw = self.getPage(f'https://box.live/boxers/{boxer1}/')
    contenido = {}
    for i in raw.find_all('div', {'class':'d-flex flex-wrap w-100'}):
      title = i.find_all('div', {'class':re.compile('stats-row__title')})
      content = i.find_all('div', {'class':re.compile('stats-row__content')})
      for a,e in zip(title,content):
        contenido[a.get_text().strip()] = (e.get_text().strip())
    contenido['boxer'] = boxer
    contenido = {key.lower(): value.lower() for key, value in contenido.items()}
    return contenido

  def update_data(self, data, temp, name, id_val=None):
    info = self.info_box_page(name)
    current_date = datetime.now()
    try:
      years_to_subtract = int(info['age'])
      new_date = current_date - timedelta(days=365 * years_to_subtract)
      birth = new_date.strftime("%Y")
      height = info['height'].split('/')[-1].replace('cm', '').strip()
      reach = info['reach'].split('/')[-1].replace('cm', '').strip()
      new_row = {'id': id_val, 'boxer': name, 'birth': birth, 'height': height, 'reach': reach,
                 'stance': info['stance'], 'box_live_data':1, 'date_of_record':current_date}
      if temp is not None:
        data.loc[temp.index, 'birth'] = birth
        data.loc[temp.index, 'height'] = height
        data.loc[temp.index, 'reach'] = reach
        data.loc[temp.index, 'stance'] = info['stance']
        data.loc[temp.index, 'box_live_data'] = 1
        data.loc[temp.index, 'date_of_record'] = current_date
        print(f'Boxer already in records, completed info: {str(data.loc[temp.index, :].values)}\n')
      else:
        if id_val is not None:
          data = pd.concat([data, pd.DataFrame([new_row])])
          print(f'New row recorded successfully: {new_row}\n')
      data.to_csv(self.filename, index=False)
      info['boxer_id'] = str(id_val)
      self.json_log(self.json_file, info)
    except KeyError:
      print(f'Key Error in retrieval, maybe no info about {name}\n')

  def auto_record_data(self, name):
    '''
    Method to aggregate another row with scrapped info.
    Parameters:
        name: str. Correct name of the boxer.
    '''
    data = pd.read_csv(self.filename)
    if name in data['boxer'].values:
      temp = data[data['boxer'] == name]
      if not pd.isna(temp['stance'].values[0]):
        print(f'Boxer already in list: {temp["boxer"].values[0]}, id: {temp["id"].values[0]}\n')
      else:
        id_val = temp['id'].values[0]
        self.update_data(data, temp, name, id_val)
    else:
      id_val = data['id'].iloc[-1] + 1
      self.update_data(data, None, name, id_val)

  def record_data(self, name, birth, height):
    '''
    Method to agregate another row manually.
    Parameters:
      name: str. Correctly name of the boxer.
      birth: int. Birth year (eg. 1991).
      height: int. Height in cm (eg. 175).
    '''
    try:
      current_date = datetime.now()
      data = pd.read_csv(self.filename).sort_values('id')
      if name in data['boxer'].values:
        temp = data[data['boxer'] == name]
        print(f'Boxer already in list\ngiven: {temp.boxer.values[0]}, id: {temp.id.values[0]}')
      else:
        id_val = data.id[-1:].values[0] + 1
        new_row = {'id':id_val, 'boxer':name, 'birth':birth, 'height':height, 'date_of_record':current_date}
        data = pd.concat([data, pd.DataFrame([new_row])])
        aut = self.authorization(self.filename, 'record data', data)
        if aut == 1:
          print(f'Row recorded successfully: {new_row}\n')
        else:
          print('Authorization to record denied')
    except Exception as e:
      print(f'Error recording name: {str(e)}\n')

  def read_data(self, row_no=0, more_rows=0):
    '''
    Method to read data in the file.
    Parameters:
      row_no: int. Number of the row to show at the beggin. Default=0
      more_rows: int. Numer of plus rows to show after the row_no. Default=0 (in this case will be all the next rows).
    Returns:
      Filtered dataframe.
    '''
    try:
      data = pd.read_csv(self.filename)
      if more_rows == 0:
        more_rows = len(data)
      elif more_rows > 0:
        more_rows = row_no + more_rows
      return data.iloc[row_no:more_rows, :]
    except Exception as e:
      print(f'Error reading data: {str(e)}\n')

  def search_name(self, name):
    '''
    Method to search for a expression in the col boxer name.
    Parameters:
      name: str. Name of the boxer or part of it to search it.
    Returns:
      Filtered dataframe.
    '''
    try:
      data = pd.read_csv(self.filename)
      temp = data[data.boxer.str.contains(name)]
      return temp
    except Exception as e:
      print(f'Error searching name: {str(e)}\n')

  def delete_rec(self, id):
    '''
    Method that delete a record.
    Parameters:
      id: int. Id of the record to delete.
    '''
    self.delete_row(self.filename, id, 'id')

## fights res

In [24]:
class Fights_results(Boxers_logger_basics):
  '''
  Class to modify, append, search or delete records in the fights results file.
  '''
  def __init__(self, filename, fights_filename, boxers_filename):
    '''
    Constructor method.
    Parameters:
      filename: str. Name of the file of the fights results.
      fights_filename: str. Name of the file where there is info. about the fight.
      boxers_filename: str. Name of the file where there is info. about the boxers.
    '''
    self.filename = filename
    self.f_filename = fights_filename
    self.b_filename = boxers_filename

  def find_existing_result(self, data, fight_id):
    if not data[(data.fight_id == fight_id)].empty:
      existing_match = data[(data.fight_id == fight_id)]['result'].values[0]
    else:
      existing_match = 0
    return existing_match

  def update_result(self, data, new_row):
    mask = data.fight_id == new_row['fight_id']
    data.loc[mask, ['result', 'endtype', 'endround']] = new_row['result'],new_row['endtype'],new_row['endround']
    return data

  def record_data_passing_values(self, fight_id, result, endtype, endround):
    '''
    Method to agregate fight result row.
    Parameters:
      fight_id: int. Id of the fight to aggregate result.
      result: str. Result of the fight [win, loss]
      endtype: str. The way the fight end [ko, desc]
      endround: int. The round the fight end.
    '''
    try:
      data = pd.read_csv(self.filename).sort_values('fight_id')
      new_row = {'result':result,'endtype':endtype,'endround':endround,'fight_id':fight_id}
      existing_res = self.find_existing_result(data, fight_id)
      if existing_res == 0:
        data = pd.concat([data, pd.DataFrame([new_row])])
      else:
        aut = input(f'The match already has result: {existing_res}\nWanna overwrite it? (y/n): ')
        if aut == 'y':
          data = self.update_result(data, new_row)
        else:
          return print('Overwrite data denied\n')
      aut = self.authorization(self.filename, 'record data', data)
      if aut == 1:
        print(f'Row recorded successfully\n')
      else:
        print('Authorization to record denied')
    except Exception as e:
      print(f'Error recording result: {str(e)}\n')

  def columns_to_conserve(self, data):
    df = data.copy()
    df = df[['result','endtype','endround','fight_id']]
    return df

  def record_data(self):
    '''
    Method to agregate fight results, via input questions.
    '''
    data = self.read_data()
    inco_data = data[data.result.isna()]
    try:
      for i in inco_data.fight_id:
        id_fight = inco_data.loc[inco_data.fight_id == i, 'fight_id'].values[0]
        boxer1_name = inco_data.loc[inco_data.fight_id == i, 'boxer1'].values[0]
        boxer2_name = inco_data.loc[inco_data.fight_id == i, 'boxer2'].values[0]
        print('=' * 20)
        print(f'Fight id: {id_fight}, between: {boxer1_name} vs {boxer2_name}\n')
        data.loc[data.fight_id == i, 'result'] = self.closer('Result (win,loss): ')
        data.loc[data.fight_id == i, 'endtype'] = self.closer('Endtype (ko,desc): ')
        data.loc[data.fight_id == i, 'endround'] = self.closer('Endround: ')
        data = self.columns_to_conserve(data)
        aut = self.authorization(self.filename, 'complete data', data)
        if aut == 1:
          print(f'Row completed successfully: {boxer1_name + " vs " + boxer2_name}\n')
        else:
          print('Authorization to record denied\n')
    except CloseException:
      print('User entered "c", exiting the program')

  def get_boxers_name(self, data):
    df = data.copy()
    boxers = pd.read_csv(self.b_filename)[['id','boxer']]
    df = df.merge(boxers, how='left', left_on='b1_id', right_on='id')
    df = df.merge(boxers, how='left', left_on='b2_id', right_on='id')
    df.drop(columns=['id_x', 'id_y'], inplace=True)
    df.rename(columns={'boxer_x':'boxer1','boxer_y':'boxer2'}, inplace=True)
    return df

  def read_data(self, row_no=0, more_rows=0, tipo='outer'):
    '''
    Method to read data in the file with some info of the fights file.
    Parameters:
      row_no: int. Number of the row to show at the beggin. Default=0
      more_rows: int. Numer of plus rows to show after the row_no. Default=0 (in this case will be all the next rows).
      tipo: str. outer (default)(show the records with and without result aggregated),
                 left (show the records with result aggregated),
                 right (show the records without result aggregated).
    Returns:
      Filtered dataframe.
    '''
    try:
      data = pd.read_csv(self.filename)
      data2 = pd.read_csv(self.f_filename)
      df = data.merge(data2, how=tipo, on='fight_id')
      df = self.get_boxers_name(df)
      df = df[['result','endtype','endround','fight_id','b1_id','boxer1','b2_id','boxer2']].sort_values('fight_id')
      if more_rows == 0:
        more_rows = len(df)
      elif more_rows > 0:
        more_rows = row_no + more_rows
      return df.iloc[row_no:more_rows, :]
    except Exception as e:
      print(f'Error reading data: {str(e)}\n')

  def search_name(self, name, boxer=1):
    '''
    Method to search for a expression in the boxers name.
    Parameters:
      name: str. Name of the boxer or part of it to search it.
      boxer: int. 1 (default)(will search in the boxer 1),
                  2 (will search in the boxer 2).
    Returns:
      Filtered dataframe.
    '''
    try:
      data = pd.read_csv(self.filename)
      data2 = pd.read_csv(self.f_filename)
      df = data.merge(data2, how='outer', on='fight_id')
      df = self.get_boxers_name(df)
      df = df[['result','endtype','endround','fight_id','b1_id','boxer1','b2_id','boxer2']]
      if boxer == 1:
        df = df[df.boxer1.str.contains(name)]
        return df
      elif boxer == 2:
        df = df[df.boxer2.str.contains(name)]
        return df
      else:
        print('Parameter boxer between 1 or 2\n')
    except Exception as e:
      print(f'Error searching name: {str(e)}\n')

  def delete_rec(self, id):
    '''
    Method that delete a record.
    Parameters:
      id: int. Id of the record to delete.
    '''
    self.delete_row(self.filename, id, 'fight_id')

## fights info

In [25]:
class Fights_info(Boxers_logger_basics):
  '''
  Class to modify, append, search or delete records in the fights info file.
  '''
  def __init__(self, filename, boxers_filename, results_filename):
    '''
    Constructor method.
    Parameters:
      filename: str. Name of the file of the fights info.
      boxers_filename: str. Name of the file where there's information about the boxers.
      results_filename: str. Name of the file with the info about the result of fights.
    '''
    self.filename = filename
    self.b_filename = boxers_filename
    self.r_filename = results_filename

  def reg_text(self, boxer_name):
    boxer = boxer_name.title().strip()
    wiki = wikipediaapi.Wikipedia('english')  # 'en' specifies the language (English in this case)
    page = wiki.page(boxer)
    text_boxer = page.text
    if ('may refer') in text_boxer:
      boxer = boxer_name.title().strip()+' '+'(boxer)'
      wiki = wikipediaapi.Wikipedia('english')
      page = wiki.page(boxer)
      text_boxer = page.text
      print('May refer founded, (boxer) agregated to the name\n')
    summary_boxer = page.summary
    len_text_boxer = len(text_boxer)
    len_summary_boxer = len(summary_boxer)
    print(f'info wiki: {page.title}\n')
    return text_boxer, summary_boxer, len_text_boxer, len_summary_boxer

  def find_existing_match(self, data, b1_id, b2_id):
    if not data[(data.b1_id == b1_id) & (data.b2_id == b2_id)].empty:
      existing_match = True
    else:
      existing_match = False
    return existing_match

  def data_to_overwirte_existing_match(self, data, b1_id, b2_id):
    id_val_exist = data[(data.b1_id == b1_id) & (data.b2_id == b2_id)].fight_id.values[0]
    data = data[~((data.b1_id == b1_id) & (data.b2_id == b2_id))]
    return id_val_exist, data

  def complete_and_append_new_row(self, data, new_row):
    b_list = pd.read_csv(self.b_filename)
    boxer1 = b_list.loc[b_list['id'] == new_row['b1_id'], 'boxer'].values[0]
    boxer2 = b_list.loc[b_list['id'] == new_row['b2_id'], 'boxer'].values[0]
    a,b,c,d = self.reg_text(boxer1)
    new_row['text_boxer1'] = a
    new_row['len_text_boxer1'] = c
    new_row['summary_boxer1'] = b
    new_row['len_summary_boxer1'] = d
    a,b,c,d = self.reg_text(boxer2)
    new_row['text_boxer2'] = a
    new_row['len_text_boxer2'] = c
    new_row['summary_boxer2'] = b
    new_row['len_summary_boxer2'] = d
    order_keys = ['fight_id', 'b1_id', 'b1_w', 'b1_wk', 'b1_d', 'b1_l', 'b1_lk', 'b1_bet',
                  'text_boxer1','len_text_boxer1','summary_boxer1','len_summary_boxer1',
                  'b2_id', 'b2_w', 'b2_wk', 'b2_d', 'b2_l', 'b2_lk', 'b2_bet',
                  'text_boxer2','len_text_boxer2','summary_boxer2','len_summary_boxer2']
    new_row2 = {k: new_row[k] for k in order_keys}
    data = pd.concat([data, pd.DataFrame([new_row2])])
    return data, boxer1, boxer2

  def save_new_data(self, file_old_data, new_data, boxer1, boxer2):
    aut = self.authorization(file_old_data, 'record data', new_data)
    if aut == 1:
      print(f'Row recorded successfully: {boxer1 + " vs " + boxer2}\n')
    else:
      print('Authorization to record denied\n')

  def record_data(self, b1_id, b1_w, b1_wk, b1_d, b1_l, b1_lk, b1_bet,
                b2_id, b2_w, b2_wk, b2_d, b2_l, b2_lk, b2_bet):
    '''
    Method to agregate fight info row.
    Parameters:
      b1 or b2 _id: int. Id of the boxer.
      b1 or b2 _w: int. Wins at the moment of the boxer.
      b1 or b2 _wk: int. Wins by ko at the moment of the boxer.
      b1 or b2 _d: int. Draws at the moment of the boxer.
      b1 or b2 _l: int. Losses at the moment of the boxer.
      b1 or b2 _lk: int. Losses by ko at the moment of the boxer.
      b1 or b2 _bet: int. Odds in decimal for the boxer in the fight.
    '''
    data = pd.read_csv(self.filename).sort_values('fight_id')
    new_row = {'b1_id':b1_id, 'b1_w':b1_w, 'b1_wk':b1_wk,
                'b1_d':b1_d, 'b1_l':b1_l, 'b1_lk':b1_lk, 'b1_bet':b1_bet,
                'b2_id':b2_id, 'b2_w':b2_w, 'b2_wk':b2_wk, 'b2_d':b2_d,
                'b2_l':b2_l, 'b2_lk':b2_lk, 'b2_bet':b2_bet}
    try:
      existing_match = self.find_existing_match(data, b1_id, b2_id)
      if existing_match == False:
        id_val = data.fight_id[-1:].values[0] + 1 # genero nuevo id
        new_row['fight_id'] = id_val
        data, boxer1, boxer2 = self.complete_and_append_new_row(data, new_row)
        self.save_new_data(self.filename, data, boxer1, boxer2)
      else: # existe match
        overwrite = input('Match already exists. Overwrite? (y/n): ')
        if overwrite.lower() != 'y': # overwrite no
          print('Overwrite denied\n')
          return 0
        else: # overwrite yes
          # bring the exisitng id and the data without that record
          new_row['fight_id'], data = self.data_to_overwrite_existing_match(
                                      existing_match, data, new_row['b1_id'],
                                      new_row['b2_id'])
          data, boxer1, boxer2 = self.complete_and_append_new_row(data, new_row)
          self.save_new_data(self.filename, data, boxer1, boxer2)
    except Exception as e:
      print(f'Error recording data: {str(e)}\n')

  def get_boxers_name(self, df):
    boxers = pd.read_csv(self.b_filename)[['id','boxer']]
    df = df.merge(boxers, how='left', left_on='b1_id', right_on='id')
    df = df.merge(boxers, how='left', left_on='b2_id', right_on='id')
    df.drop(columns=['id_x', 'id_y'], inplace=True)
    df.rename(columns={'boxer_x':'boxer1','boxer_y':'boxer2'}, inplace=True)
    return df

  def read_data(self, row_no=0, more_rows=0):
    '''
    Method to read data in the file with names of the boxers.
    Parameters:
      row_no: int. Number of the row to show at the beggin. Default=0
      more_rows: int. Numer of plus rows to show after the row_no. Default=0 (in this case will be all the next rows).
    Returns:
      Filtered dataframe.
    '''
    try:
      data = pd.read_csv(self.filename)
      df = self.get_boxers_name(data)
      df = df[['fight_id','b1_id','boxer1','b1_w','b1_wk','b1_d',
               'b1_l','b1_lk','b1_bet','len_text_boxer1',
               'b2_id','boxer2','b2_w','b2_wk','b2_d','b2_l',
               'b2_lk','b2_bet','len_text_boxer2']]
      if more_rows == 0:
        more_rows = len(df)
      elif more_rows > 0:
        more_rows = row_no + more_rows
      return df.iloc[row_no:more_rows, :]
    except Exception as e:
      print(f'Error reading data: {str(e)}\n')

  def search_name(self, name, boxer=1):
    '''
    Method to search for a expression in the boxers name.
    Parameters:
      name: str. Name of the boxer or part of it to search it.
      boxer: int. 1 (default)(will search in the boxer 1),
                  2 (will search in the boxer 2).
    Returns:
      Filtered dataframe.
    '''
    try:
      df = pd.read_csv(self.filename)
      df = self.get_boxers_name(df)
      df = df[['fight_id','b1_id','boxer1','b1_w','b1_wk','b1_d','b1_l','b1_lk','b1_bet','len_text_boxer1',
                   'b2_id','boxer2','b2_w','b2_wk','b2_d','b2_l','b2_lk','b2_bet','len_text_boxer2']]
      if boxer == 1:
        df = df[df.boxer1.str.contains(name)]
        return df
      elif boxer == 2:
        df = df[df.boxer2.str.contains(name)]
        return df
      else:
        print('Parameter boxer between 1 or 2\n')
    except Exception as e:
      print(f'Error searching name: {str(e)}\n')

  def delete_rec(self, id):
    '''
    Method that delete a record, will also delete the fights results file rec with that id.
    Parameters:
      id: int. Id of the record to delete in both fight info and fight results files.
    '''
    self.delete_row(self.filename, id, 'fight_id')
    self.delete_row(self.r_filename, id, 'fight_id')

## fights from browse_ai

In [26]:
class Fights_browse_ai(Fights_info, Boxers_info):
  '''
  Class to agregate, complete, search or delete records in
  the fights info file according to the data obtained from browse ai.
  '''
  def __init__(self, fights_filename, boxers_filename, json_file, figths_extracted_link, fights_extracted_sheet):
    '''
    Cosntructor method.
    Parameters:
      fights_filename: str. Name of the file where there is information about the fights.
      boxers_filename: str. Name of the file where there is information about the boxers.
      json_filename: str. Name of the json file with scrapped data.
      fights_extracted_link: str. Link to the book with the info extracted from caliente.
      fights_extracted_sheet: str. Name of the sheet with the info extracted from caliente.
    '''
    self.f_filename = fights_filename
    self.b_filename = boxers_filename
    self.j_file = json_file
    self.f_link = figths_extracted_link
    self.f_sheet = fights_extracted_sheet

  def data_from_f_sheet(self):
    bai_df = read_gsheet_shared(self.f_link, self.f_sheet)
    return bai_df

  def first_process(self, data):
    df = data.copy()
    df.columns = ['date_extract','remove','remove1','remove2','remove3','fight_date','boxer1','b1_bet','d_bet','boxer2','b2_bet']
    df.drop(columns=['remove','remove1','remove2','remove3'], inplace=True)
    df['fight_date'] = [i.replace('ene','1').replace('feb','2').replace('mar','3').replace('abr','4')\
                              .replace('may','5').replace('jun','6').replace('jul','7').replace('sep','9')\
                              .replace('oct','10').replace('nov','11').replace('dic','12') for i in df.fight_date]
    df['date_extract'] = pd.to_datetime(df['date_extract'])
    df['year_fight'] = df.date_extract.dt.year
    df['current_month'] = df.date_extract.dt.month
    df['month_fight'] = [int(i.split()[-1]) for i in df.fight_date]
    df['day_fight'] = [int(i.split()[0]) for i in df.fight_date]
    df.loc[df.current_month > df.month_fight, 'year_fight'] = df.loc[df.current_month > df.month_fight, 'year_fight'] + 1
    df['date_fight'] = df.year_fight.astype(str) + '-' + df.month_fight.astype(str) + '-' + df.day_fight.astype(str)
    df['date_fight'] = pd.to_datetime(df.date_fight)
    df['boxer1'] = df.boxer1.str.lower()
    df['boxer2'] = df.boxer2.str.lower()
    df.drop(columns=['fight_date','year_fight','month_fight','day_fight','current_month'], inplace=True)
    return df

  # def first_process(self, data, current_year):
  #   bai = data.drop(columns='Position')
  #   bai['date'] = [i.replace('ene','1').replace('feb','2').replace('mar','3').replace('abr','4')\
  #                             .replace('may','5').replace('jun','6').replace('jul','7').replace('sep','9')\
  #                             .replace('oct','10').replace('nov','11').replace('dic','12') for i in bai.date]
  #   count = 0
  #   anio = current_year
  #   lista_anio = []
  #   for i in bai.date.str.split('-'):
  #     mes = i[-1]
  #     if count == 0:
  #       lista_anio.append(anio)
  #     else:
  #       if mes_anterior > mes:
  #         anio += 1
  #         lista_anio.append(anio)
  #       else:
  #         lista_anio.append(anio)
  #     count += 1
  #     mes_anterior = mes

  #   bai['date'] = bai.date + '-' + pd.Series(map(str,lista_anio))
  #   bai['date'] = pd.to_datetime(bai.date, dayfirst=True)
  #   bai['boxer1'] = bai.boxer1.str.lower()
  #   bai['boxer2'] = bai.boxer2.str.lower()
  #   bai.rename(columns={'date':'c_f'}, inplace=True)
  #   return bai

  def check_existing_records(self, data):
    fights = pd.read_csv(self.f_filename)
    fight_class = Fights_info(self.f_filename, self.b_filename)
    fights = fight_class.get_boxers_name(fights)
    fights = fights[['fight_id','boxer1','boxer2']]
    df = data.merge(fights, how='left', on=['boxer1', 'boxer2'])
    df = df[df.fight_id.isna()].drop(columns=['fight_id'])
    return df

  def setting_ids(self, data):
    fights = pd.read_csv(self.f_filename)
    fight_class = Fights_info(self.f_filename, self.b_filename)
    fights = fight_class.get_boxers_name(fights)
    fights = fights[['fight_id','boxer1','boxer2']]
    res = fights.merge(data, how='outer', on=['boxer1','boxer2'])
    res.sort_values('fight_id', inplace=True)
    last_id = res.fight_id.max()
    res2 = res[res.fight_id.isna()]
    for i in res2.index:
      last_id += 1
      res2.loc[res2.index == i, 'fight_id'] = last_id
    return res2

  def get_boxers_ids(self, boxer_names):
    box = pd.read_csv(self.b_filename)
    lista = []
    for i in boxer_names:
      try:
        id = box.loc[box.boxer == i, 'id'].values[0]
        lista.append(id)
      except IndexError:
        boxers = Boxers_info(self.b_filename, self.j_file)
        try:
          boxers.auto_record_data(i)
          box = pd.read_csv(self.b_filename)
          id = box.loc[box.boxer == i, 'id'].values[0]
          lista.append(id)
        except IndexError:
          print(f'Name: {i}')
          boxers.record_data(i, # nombre del boxeador
                             input('Birth: '),
                             input('height: '))
          box = pd.read_csv(self.b_filename)
          id = box.loc[box.boxer == i, 'id'].values[0]
          lista.append(id)
    return lista

  def get_wiki_info(self, boxer_names):
    boxers = boxer_names
    text_boxer1 = []
    len_text_boxer1 = []
    summary_boxer1 = []
    len_summary_boxer1 = []
    for i in [i for i in boxers]:
      a,b,c,d = self.reg_text(i)
      text_boxer1.append(a)
      len_text_boxer1.append(c)
      summary_boxer1.append(b)
      len_summary_boxer1.append(d)
    return text_boxer1, len_text_boxer1, summary_boxer1, len_summary_boxer1

  def columns_to_conserve(self, data):
    data = data.copy()
    data = data[['fight_id', 'b1_id', 'b1_w', 'b1_wk', 'b1_d', 'b1_l', 'b1_lk', 'b1_bet',
       'text_boxer1', 'summary_boxer1', 'len_text_boxer1',
       'len_summary_boxer1', 'b2_id', 'b2_w', 'b2_wk', 'b2_d', 'b2_l', 'b2_lk',
       'b2_bet', 'text_boxer2', 'summary_boxer2', 'len_text_boxer2',
       'len_summary_boxer2', 'date_fight', 'date_extract', 'd_bet']]
    return data

  def auto_record_data(self):
    '''
    Method to get the extracted fights and odds from casino info.
    Parameters:
      current_year: int. eg. 2023.
    '''
    casino_data = self.data_from_f_sheet()
    casino_data = self.first_process(casino_data)
    casino_data = self.check_existing_records(casino_data)
    casino_data = self.setting_ids(casino_data)
    casino_data['b1_id'] = self.get_boxers_ids(casino_data.boxer1)
    casino_data['b2_id'] = self.get_boxers_ids(casino_data.boxer2)
    a,b,c,d = self.get_wiki_info(casino_data.boxer1)
    casino_data['text_boxer1'] = a
    casino_data['len_text_boxer1'] = b
    casino_data['summary_boxer1'] = c
    casino_data['len_summary_boxer1'] = d
    a,b,c,d = self.get_wiki_info(casino_data.boxer2)
    casino_data['text_boxer2'] = a
    casino_data['len_text_boxer2'] = b
    casino_data['summary_boxer2'] = c
    casino_data['len_summary_boxer2'] = d
    data_existent = pd.read_csv(self.f_filename).sort_values('fight_id')
    data = pd.concat([data_existent, casino_data])
    data = self.columns_to_conserve(data)
    aut = self.authorization(self.f_filename, 'auto record data', data)
    if aut == 1:
      print(f'Auto record data successfully\n')
    else:
      print('Authorization to record denied\n')

  def complete_records(self):
    '''
    Method to complete fight info records manually, with stats from boxers info.
    '''
    data = pd.read_csv(self.f_filename).sort_values('fight_id')
    data_names = self.get_boxers_name(data)
    inco_data = data_names[(data_names.b1_w.isna()) & (data_names.b2_w.isna())]
    try:
      for i in inco_data.fight_id:
        id_fight = inco_data.loc[inco_data.fight_id == i, 'fight_id'].values[0]
        boxer1_name = inco_data.loc[inco_data.fight_id == i, 'boxer1'].values[0]
        boxer2_name = inco_data.loc[inco_data.fight_id == i, 'boxer2'].values[0]
        print('=' * 20)
        print(f'Fight id: {id_fight}, between: {boxer1_name} vs {boxer2_name}\n')
        data.loc[data.fight_id == i, 'b1_w'] = self.closer('Wins boxer1: ')
        data.loc[data.fight_id == i, 'b1_wk'] = self.closer('Ko wins boxer1: ')
        data.loc[data.fight_id == i, 'b1_d'] = self.closer('Draws boxer1: ')
        data.loc[data.fight_id == i, 'b1_l'] = self.closer('Loss boxer1: ')
        data.loc[data.fight_id == i, 'b1_lk'] = self.closer('Ko loss boxer1: ')
        data.loc[data.fight_id == i, 'b2_w'] = self.closer('Wins boxer2: ')
        data.loc[data.fight_id == i, 'b2_wk'] = self.closer('Ko wins boxer2: ')
        data.loc[data.fight_id == i, 'b2_d'] = self.closer('Draws boxer2: ')
        data.loc[data.fight_id == i, 'b2_l'] = self.closer('Loss boxer2: ')
        data.loc[data.fight_id == i, 'b2_lk'] = self.closer('Ko loss boxer2: ')
        aut = self.authorization(self.f_filename, 'complete data', data)
        if aut == 1:
          print(f'Row completed successfully: {boxer1_name + " vs " + boxer2_name}\n')
        else:
          print('Authorization to record denied\n')
    except CloseException:
      print('User entered "c", exiting the program')

## examples

In [27]:
# help(Fights_info.__init__)

Help on function __init__ in module __main__:

__init__(self, filename, boxers_filename, results_filename)
    Constructor method.
    Parameters:
      filename: str. Name of the file of the fights info.
      boxers_filename: str. Name of the file where there's information about the boxers.
      results_filename: str. Name of the file with the info about the result of fights.



In [28]:
# %xmode verbose
# import pdb
# # pdb.set_trace()
# f_i = Fights_info('/content/fights_info.csv',
#                   '/content/boxers_info.csv',
#                   '/content/fights_results.csv')

In [35]:
# f_i.read_data()

,fight_id,b1_id,boxer1,b1_w,b1_wk,b1_d,b1_l,b1_lk,b1_bet,len_text_boxer1,b2_id,boxer2,b2_w,b2_wk,b2_d,b2_l,b2_lk,b2_bet,len_text_boxer2
0,0,1,gervonta davis,50,32,1,2,0,1.2,NaN,2,dimitry bivol,30,20,0,1,0,2.5,NaN
1,1,0,canelo alvarez,50,32,1,2,0,1.2,NaN,1,gervonta davis,30,20,0,1,0,2.5,NaN
2,35,2,dimitry bivol,50,32,1,2,0,1.2,NaN,0,canelo alvarez,30,20,0,1,0,1.1,NaN
3,37,2,dimitry bivol,50,32,1,2,0,1.2,NaN,0,canelo alvarez,30,20,0,1,0,1.1,NaN


In [36]:
# pd.read_csv('fights_info.csv')

,fight_id,b1_id,b1_w,b1_wk,b1_d,b1_l,b1_lk,b1_bet,text_boxer1,summary_boxer1,...,b2_w,b2_wk,b2_d,b2_l,b2_lk,b2_bet,text_boxer2,summary_boxer2,len_text_boxer2,len_summary_boxer2
0,0,1,50,32,1,2,0,1.2,NaN,NaN,...,30,20,0,1,0,2.5,NaN,NaN,NaN,NaN
1,1,0,50,32,1,2,0,1.2,NaN,NaN,...,30,20,0,1,0,2.5,NaN,NaN,NaN,NaN
2,35,2,50,32,1,2,0,1.2,NaN,NaN,...,30,20,0,1,0,1.1,NaN,NaN,NaN,NaN
3,37,2,50,32,1,2,0,1.2,NaN,NaN,...,30,20,0,1,0,1.1,NaN,NaN,NaN,NaN


In [37]:
# pd.read_csv('/content/fights_results.csv')

,result,endtype,endround,fight_id
0,win,dec,12,0
1,win,ko,7,1
2,loss,dec,12,35


In [34]:
# f_i.delete_rec(2)

Proceed with delete record? (y/n):y
Row deleted successfully

Proceed with delete record? (y/n):y
Row deleted successfully



In [ ]:
# complet = Fights_browse_ai('/content/fight_info_real.csv',
#                  '/content/boxers_info_.csv',
#                  '/content/boxers_json_data.json',
#                  '13uCE8C95ZFd4chQowzSFb_okVT1ikn3OPFg4geTFTVw',
#                  'caliente_box_fights')

In [ ]:
# complet.auto_record_data()

New row recorded successfully: {'id': 6579, 'boxer': 'souleymane cissokho', 'birth': '1991', 'height': '179', 'reach': '179', 'stance': 'orthodox (r)', 'box_live_data': 1, 'date_of_record': datetime.datetime(2023, 11, 13, 18, 38, 36, 198268)}

Json data appended to /content/boxers_json_data.json

Key Error in retrieval, maybe no info about julissa alejandra guzman

Name: julissa alejandra guzman
Birth: 1222
height: 122
Proceed with record data? (y/n):y
Row recorded successfully: {'id': 6580, 'boxer': 'julissa alejandra guzman', 'birth': '1222', 'height': '122', 'date_of_record': datetime.datetime(2023, 11, 13, 18, 38, 53, 257724)}

New row recorded successfully: {'id': 6581, 'boxer': 'sivenathi nontshinga', 'birth': '1999', 'height': '165', 'reach': '170', 'stance': 'orthodox (r)', 'box_live_data': 1, 'date_of_record': datetime.datetime(2023, 11, 13, 18, 38, 57, 313218)}

Json data appended to /content/boxers_json_data.json

Key Error in retrieval, maybe no info about antonio mireles



In [ ]:
# complet.complete_records()

Fight id: 80.0, between: souleymane cissokho vs isaias lucero

Wins boxer1: 1
Ko wins boxer1: 1
Draws boxer1: 1
Loss boxer1: 1
Ko loss boxer1: 1
Wins boxer2: 1
Ko wins boxer2: 1
Draws boxer2: 1
Loss boxer2: 1
Ko loss boxer2: 1
Proceed with complete data? (y/n):y
Row completed successfully: souleymane cissokho vs isaias lucero

Fight id: 81.0, between: julissa alejandra guzman vs ramla ali

Wins boxer1: 1
Ko wins boxer1: 1
Draws boxer1: 1
Loss boxer1: 1
Ko loss boxer1: 1
Wins boxer2: 1
Ko wins boxer2: 1
Draws boxer2: 1
Loss boxer2: 1
Ko loss boxer2: 1
Proceed with complete data? (y/n):y
Row completed successfully: julissa alejandra guzman vs ramla ali

Fight id: 82.0, between: michal cieslak vs tommy mccarthy

Wins boxer1: 1
Ko wins boxer1: 1
Draws boxer1: 
Loss boxer1: 1
Ko loss boxer1: 1
Wins boxer2: 1
Ko wins boxer2: 
Draws boxer2: 
Loss boxer2: 
Ko loss boxer2: 
Proceed with complete data? (y/n):y
Row completed successfully: michal cieslak vs tommy mccarthy

Fight id: 83.0, between:

In [ ]:
# fig = pd.read_csv('/content/fight_info_real.csv')
# box = pd.read_csv('/content/boxers_info_.csv')
# ja = pd.read_json('/content/boxers_json_data.json')
# # res = pd.read_csv('/content/fights_results.csv')

In [ ]:
# ja.columns

Index(['age', 'height', 'reach', 'stance', 'wins', 'by ko', 'ko%', 'lost',
       'stopped', 'draws', 'debut', 'pro rds', 'boxer',
       'the ring cruiserweight champion', 'next scheduled defence',
       'last defence', 'time inactive', 'won against', 'time held',
       'total defences', 'activity average', 'mandatory challenger',
       'last mandatory', 'since a mandatory',
       'ibf world cruiserweight champion', 'wba world featherweight champion',
       'ring', 'wbo', 'ibf', 'wbc', 'wba', 'wbo world flyweight champion',
       'wbo world featherweight champion', 'ibf world flyweight champion',
       'wbo world junior featherweight champion',
       'wbc world super bantamweight champion',
       'wbc world bantamweight champion', 'bbboc british heavyweight champion',
       'boxer_id', 'ibf world junior lightweight champion',
       'wbc world super featherweight champion',
       'wbc world cruiserweight champion',
       'bbboc british middleweight champion',
       'bbboc

In [ ]:
# results = Fights_results('fights_results.csv',
#                          'fights_info.csv',
#                          'boxers_info.csv')

In [ ]:
# results.read_data()

,result,endtype,endround,fight_id,b1_id,boxer1,b2_id,boxer2
0,win,dec,12.0,0,1,gervonta davis,2,dimitry bivol
1,win,ko,7.0,1,0,canelo alvarez,1,gervonta davis
2,win,ko,8.0,2,2,dimitry bivol,0,canelo alvarez
3,loss,dec,12.0,35,2,dimitry bivol,0,canelo alvarez
4,1,1,12.0,36,2,dimitry bivol,0,canelo alvarez
5,1,0,12.0,37,2,dimitry bivol,0,canelo alvarez


In [ ]:
# results.record_data()

Fight id: 37, between: dimitry bivol vs canelo alvarez

Result (win,loss): 1
Endtype (ko,desc): 0
Endround: 12
Proceed with complete data? (y/n):y
Row completed successfully: dimitry bivol vs canelo alvarez

